In [81]:
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup

from tqdm.notebook import tqdm 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [83]:
chrome_options = webdriver.ChromeOptions()
chrome_path = "C:\Program Files\Google\Chrome\Application\chrome.exe"  # 실제 경로
chrome_options.binary_location = chrome_path

driver_path = "chromedriver.exe"
myservice = Service(driver_path)
driver = webdriver.Chrome(service=myservice, options=chrome_options)

print(type(driver))

wait_time = 10
driver.implicitly_wait(wait_time)

<class 'selenium.webdriver.chrome.webdriver.WebDriver'>


In [74]:
driver.maximize_window()

In [84]:
starbucks_url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(starbucks_url)

In [85]:
# 스타벅스 서울 링크 클릭
starbucks_seoul_selector = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
seoul_link = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_seoul_selector)))
seoul_link.click()

In [86]:
# 서울 > 전체 링크 클릭
starbucks_seoul_all = '#mCSB_2_container > ul > li:nth-child(1) > a'
seoul_all_link = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_seoul_all)))
seoul_all_link.click()

In [92]:
# 서울 > 전체 링크 > html 파일생성
html = driver.page_source
filename = "file/starbucks.html"
htmlFile = open(filename, mode='w', encoding='utf-8')
print(html, file=htmlFile)
htmlFile.close()
print(filename + "파일 생성됨")

starbucks.html파일 생성됨


In [93]:
soup = BeautifulSoup(html, 'html.parser')
print(type(soup))

<class 'bs4.BeautifulSoup'>


In [94]:
# 서울 전체 매장 li 찾기
container = soup.find('div', id='mCSB_3_container')
storeAll = container.find_all('li')
print(f'서울 전체 매장 개수 : {len(storeAll)}')

서울 전체 매장 개수 : 660


In [95]:
# 서울 전체 li html로 만들기
html_partial = '\n\n'.join(str(li) for li in storeAll)
htmlFile = open('file/mCSB_3_container_li.html', mode='w', encoding='utf-8')
print(html_partial, file=htmlFile)
htmlFile.close()
print('파일 생성!')

파일 생성!


In [108]:
# 매장이름 / 주소전체 +(구 포함) / 위도 경도

starbuckusData = []
address = []

for li in storeAll:
    name = li.get('data-name')
    lat = li.get('data-lat')
    long = li.get('data-long')
    address = li.find('p','result_details').getText().replace("1522-3232", "")
    gu = address.split(' ')[1]
    starbuckusData.append(["스타벅스",name,address, gu, lat, long, ])
print(starbuckusData[:1])

[['스타벅스', '역삼아레나빌딩', '서울특별시 강남구 언주로 425 (역삼동)', '강남구', '37.501087', '127.043069']]


In [109]:
htmlFile = open('file/starbuckusData.html', mode='w', encoding='utf-8')
print(starbuckusData, file=htmlFile)
htmlFile.close()
print('파일 생성!')

파일 생성!


In [110]:
sbDataFrame = pd.DataFrame(starbuckusData, columns=['브랜드','지점명', '주소', '군구', '위도', '경도'])
sbDataFrame.head()

,브랜드,지점명,주소,군구,위도,경도
0,스타벅스,역삼아레나빌딩,서울특별시 강남구 언주로 425 (역삼동),강남구,37.501087,127.043069
1,스타벅스,논현역사거리,서울특별시 강남구 강남대로 538 (논현동),강남구,37.510178,127.022223
2,스타벅스,신사역성일빌딩,서울특별시 강남구 강남대로 584 (논현동),강남구,37.5139309,127.0206057
3,스타벅스,국기원사거리,서울특별시 강남구 테헤란로 125 (역삼동),강남구,37.499517,127.031495
4,스타벅스,대치재경빌딩,서울특별시 강남구 남부순환로 2947 (대치동),강남구,37.494668,127.062583


In [111]:
fileName = 'file/starbucksFile.csv'
sbDataFrame.to_csv(fileName, index=False)
print('starbucksFile 파일 생성')

starbucksFile 파일 생성
